# Cache $

In [1]:
import pandas as pd

#added this because it's indicated as useful but not imported
import urllib
from zipfile import ZipFile
import os

from IPython.display import display

I Definitely copy and pasted this next piece. I found it here:
http://stackoverflow.com/a/13895723

In [2]:
def reporthook(blocknum, blocksize, totalsize):
    readsofar = blocknum * blocksize
    if totalsize > 0:
        percent = readsofar * 1e2 / totalsize
        s = "\r%5.1f%% %*d / %d" % (
            percent, len(str(totalsize)), readsofar, totalsize)
        os.sys.stderr.write(s)
        if readsofar >= totalsize: # near the end
            os.sys.stderr.write("\n")
    else: # total size is unknown
        os.sys.stderr.write("read %d\n" % (readsofar,))

In [3]:
datadir = "data"
try:
    os.makedirs(datadir)
except:
    pass #directory already exists. :P

#my attempt at data caching
#it tries to open a file int the data directory.
#if the file does not exist, it downloads it and tries to open it again

#Input: a list of links to the data files.
#Output: a list of dataframes
def get_cached_or_dl(url_list):
    theList = []
    for url in url_list:
        filename = os.path.join(datadir,url.split("/")[-1])
        extension = filename.split(".")[-1].lower()
        try:
            if(extension == "zip"):
                zipF = ZipFile(filename)
                #Let's try opening every file in the zip archive
                data = []
                for f in zipF.namelist():
                    fileinzip = os.path.join(datadir,f)
                    extinzip = f.split(".")[-1]
                    if(extinzip == "csv"):
                        data.append(pd.read_csv(fileinzip))
                    elif(extinzip == "xls" or extinzip == "xlsx"):
                        data.append(pd.read_excel(fileinzip))
                    elif(extinzip == "json"):
                        data.append(pd.read_json(fileinzip))
                    else:
                        print("how to open " + fileinzip + " in zip?")
            elif(extension == "csv"):
                data = [pd.read_csv(filename)]
            elif(extension == "xls" or extension == "xlsx"):
                data = [pd.read_excel(filename)]
            elif(extension == "json"):
                data = [pd.read_json(filename)]
            else:
                data = []
                print("how to open " + filename + "?")
        except FileNotFoundError as e:
            #if file doesn't exist, download it and try opening it again
            if(extension == "zip"):
                try:
                    zipF = ZipFile(filename)
                except FileNotFoundError as e:
                    #the zip file needs to be downloaded
                    urllib.request.urlretrieve(url, filename, reporthook)
                    zipF = ZipFile(filename)
            else:
                urllib.request.urlretrieve(url, filename, reporthook)
            
            if(extension == "zip"):
                #Let's extract and then open the files
                zipF.extractall(datadir)
                data = []
                for f in zipF.namelist():
                    fileinzip = os.path.join(datadir,f)
                    extinzip = f.split(".")[-1]
                    if(extinzip == "csv"):
                        data.append(pd.read_csv(fileinzip))
                    elif(extinzip == "xls" or extinzip == "xlsx"):
                        data.append(pd.read_excel(fileinzip))
                    elif(extinzip == "json"):
                        data.append(pd.read_json(fileinzip))
                    else:
                        print("how to open " + fileinzip + " in zip?")
            elif(extension == "csv"):
                data = [pd.read_csv(filename)]
            elif(extension == "xls" or extension == "xlsx"):
                data = [pd.read_excel(filename)]
            elif(extension == "json"):
                data = [pd.read_json(filename)]
        theList.extend(data)
    return theList

### Some usage

In [4]:
some_data_files = ["https://www.huduser.gov/portal/datasets/fmr/fmr2016p/asq_zctadata_2013_00000to19999.zip",
                 "https://www.huduser.gov/portal/datasets/fmr/fmr2017/FY2017_hypothetical_safmrs.xlsx"]

In [5]:
theList = get_cached_or_dl(some_data_files)

100.0% 32710656 / 32705577
100.5% 1097728 / 1092364


In [6]:
#the return was a list of dataframes
for df in theList:
    display(df.head())

,zcta,geoid,name,tblid,stub,num_estimate,margin_error,count_40p,actual_count
0,601,H2500US00601,ZCTA5 00601,S5413T,No bedroom,0,20,0.0,0.0
1,601,H2500US00601,ZCTA5 00601,S5413T,With cash rent:,0,20,NaN,NaN
2,601,H2500US00601,ZCTA5 00601,S5413T,Less than $100,0,20,NaN,NaN
3,601,H2500US00601,ZCTA5 00601,S5413T,$100 to $149,0,20,NaN,NaN
4,601,H2500US00601,ZCTA5 00601,S5413T,$150 to $199,0,20,NaN,NaN


,zip_code,metro_code,metro_name,area_rent_br0,area_rent_br1,area_rent_br2,area_rent_br3,area_rent_br4
0,76437,10180,"Abilene, TX MSA",470,510,680,850,1130
1,76443,10180,"Abilene, TX MSA",470,510,680,850,1130
2,76464,10180,"Abilene, TX MSA",470,510,680,850,1130
3,76469,10180,"Abilene, TX MSA",500,550,730,910,1220
4,79501,10180,"Abilene, TX MSA",470,510,680,850,1130
